# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sys

import pandas as pd
import numpy as np
import nltk
import re

nltk.download(['punkt', 'wordnet','stopwords', 'averaged_perceptron_tagger'])

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.externals import joblib

from sqlalchemy import create_engine
import pickle

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report,accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///Disaster_Messages.db')
df = pd.read_sql("SELECT * FROM Disaster_Messages", engine)
X = df['message']
Y = df.iloc[:,4:]

In [3]:
# Get category names
category_names = list(df.columns[4:])

### 2. Write a tokenization function to process your text data

In [13]:
def tokenize(text):
    # Remove punctuation and convert text to lower case
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    text = text.lower()
    
    # Lemmatize words
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    cleaned_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        cleaned_tokens.append(clean_tok)
    
    # Remove stop-words
    stop_words = stopwords.words("english")
    
    cleaned_tokens = [word for word in tokens if word not in stop_words]
    
    return cleaned_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
# Initialize a pipeline with Count Vectorizer, TF IDF transformer and Random Forest Classifier
# After experimentation observed that AdaBoost Classifier has better performance; hence, used that in final deployment
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
# Train test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size = 0.2, random_state = 42)

# Pipeline fit
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
# Predict using the pipeline
Y_pred = pipeline.predict(X_test)

# Calculate overall accuracy
Overall_acc = (Y_pred == Y_test).mean().mean()

print('Overall accuracy {0:.2f}% \n'.format(Overall_acc*100))

Overall accuracy 94.45% 



In [8]:
# Get classification report results
Y_pred_DF = pd.DataFrame(Y_pred, columns = Y_test.columns)
for column in Y_test.columns:
    print('****     *****     ***** \n')
    print('Feature Name: {}\n'.format(column))
    print(classification_report(Y_test[column],Y_pred_DF[column]))

****     *****     ***** 

Feature Name: related

             precision    recall  f1-score   support

          0       0.63      0.47      0.54      1266
          1       0.84      0.91      0.87      3938
          2       0.50      0.17      0.26        40

avg / total       0.78      0.80      0.79      5244

****     *****     ***** 

Feature Name: request

             precision    recall  f1-score   support

          0       0.89      0.97      0.93      4349
          1       0.77      0.43      0.55       895

avg / total       0.87      0.88      0.87      5244

****     *****     ***** 

Feature Name: offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      5244

****     *****     ***** 

Feature Name: aid_related

             precision    recall  f1-score   support

          0       0.75      0.85      0.80    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f72fd4ac598>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [10]:
# GridSearchCV
parameters = {'vect__min_df': [1, 5],
              'clf__estimator__max_depth': [10, 50],
              'clf__estimator__min_samples_leaf':[1, 10]}


cv = GridSearchCV(pipeline, parameters)

In [11]:
model=cv.fit(X_train, Y_train)

In [12]:
cv.best_estimator_

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
# Testing the model post GridSearchCV
# The accuracy drops surprisingly, default RF classifier params seems better than the GridSearchCV params randomly feeded
Y_pred = model.predict(X_test)

Overall_acc = (Y_pred == Y_test).mean().mean()

print('Overall accuracy {0:.2f}% \n'.format(Overall_acc*100))

Overall accuracy 94.19% 



In [14]:
# Testing the model post GridSearchCV
Y_pred_DF = pd.DataFrame(Y_pred, columns = Y_test.columns)
for column in Y_test.columns:
    print('****     *****     ***** \n')
    print('Feature Name: {}\n'.format(column))
    print(classification_report(Y_test[column],Y_pred_DF[column]))

****     *****     ***** 

Feature Name: related

             precision    recall  f1-score   support

          0       0.73      0.17      0.27      1266
          1       0.78      0.98      0.87      3938
          2       0.00      0.00      0.00        40

avg / total       0.76      0.78      0.72      5244

****     *****     ***** 

Feature Name: request

             precision    recall  f1-score   support

          0       0.88      0.99      0.93      4349
          1       0.84      0.35      0.49       895

avg / total       0.87      0.88      0.85      5244

****     *****     ***** 

Feature Name: offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.00      0.00      0.00        26

avg / total       0.99      0.99      0.99      5244

****     *****     ***** 

Feature Name: aid_related

             precision    recall  f1-score   support

          0       0.75      0.88      0.81    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [15]:
# Trying AdaBoost Classifier
pipeline_new = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

# Model fit
pipeline_new.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [19]:
pipeline_new.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f72fd4ac598>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_rang

In [16]:
# Testing the AdaBoost Classifier model
Y_pred = pipeline_new.predict(X_test)

Overall_acc = (Y_pred == Y_test).mean().mean()

print('Overall accuracy {0:.2f}% \n'.format(Overall_acc*100))

Overall accuracy 94.71% 



In [17]:
Y_pred_DF = pd.DataFrame(Y_pred, columns = Y_test.columns)
for column in Y_test.columns:
    print('****     *****     ***** \n')
    print('Feature Name: {}\n'.format(column))
    print(classification_report(Y_test[column],Y_pred_DF[column]))

****     *****     ***** 

Feature Name: related

             precision    recall  f1-score   support

          0       0.68      0.08      0.15      1266
          1       0.77      0.99      0.86      3938
          2       0.36      0.12      0.19        40

avg / total       0.74      0.76      0.68      5244

****     *****     ***** 

Feature Name: request

             precision    recall  f1-score   support

          0       0.90      0.97      0.94      4349
          1       0.78      0.50      0.61       895

avg / total       0.88      0.89      0.88      5244

****     *****     ***** 

Feature Name: offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.00      0.00      0.00        26

avg / total       0.99      0.99      0.99      5244

****     *****     ***** 

Feature Name: aid_related

             precision    recall  f1-score   support

          0       0.76      0.87      0.81    

### 9. Export your model as a pickle file

In [18]:
pickle.dump(cv, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [28]:
# Testing the model on one example
model = joblib.load("model.pkl")
classification_labels = model.predict(['Some 2,000 women protesting against the conduc'])[0]
classification_results = dict(zip(df.columns[4:], classification_labels))

In [29]:
classification_results 

{'related': 1,
 'request': 0,
 'offer': 0,
 'aid_related': 1,
 'medical_help': 0,
 'medical_products': 0,
 'search_and_rescue': 0,
 'security': 0,
 'military': 0,
 'child_alone': 0,
 'water': 0,
 'food': 0,
 'shelter': 0,
 'clothing': 0,
 'money': 0,
 'missing_people': 0,
 'refugees': 0,
 'death': 0,
 'other_aid': 0,
 'infrastructure_related': 0,
 'transport': 0,
 'buildings': 0,
 'electricity': 0,
 'tools': 0,
 'hospitals': 0,
 'shops': 0,
 'aid_centers': 0,
 'other_infrastructure': 0,
 'weather_related': 0,
 'floods': 0,
 'storm': 0,
 'fire': 0,
 'earthquake': 0,
 'cold': 0,
 'other_weather': 0,
 'direct_report': 0}